<a href="https://colab.research.google.com/github/liavaldetaro/RadarDataset/blob/main/RadarDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Radar dataset**

In this notebook, we will explore a radar dataset from Kaggle: https://www.kaggle.com/datasets/iroldan/real-doppler-raddar-database


> The radar used to capture the data is a ubiquitous or persistent radar system developed by the Microwave and Radar Group from the UPM called RAD-DAR (Digital Array Receiver). The radar uses a frequency-modulated continuous wave (FMCW) on a frequency band centered at 8.75 GHz with a maximum bandwidth of 500 MHz.

The data contains **labelled** radar data from cars, birds and drones. Let's try to see if we can classify it!



In [4]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections.abc import Iterator
import tensorflow as tf
from keras.callbacks import EarlyStopping
import kagglehub

In [2]:

# Let's start by downloading the dataset into our notebook:
path = kagglehub.dataset_download("iroldan/real-doppler-raddar-database")

# Let's print the path to where the data is saved to:
print("Path to dataset files:", path)

100%|██████████| 63.6M/63.6M [00:00<00:00, 209MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/iroldan/real-doppler-raddar-database/versions/2


## **Data analysis**

Perfect! Our data is loaded into the .cache folder. Let's start by taking a look at what the data looks like.

In [ ]:
car_image = pd.read_csv(

)

Now that we are a bit more familiar with the data, we can start **preparing it** to feed into our AI model.
There are a couple of steps to this preparation:

1.  Load in the images and corresponding labels (if it is a car, a drone or a bird)
2.  Normalize our data
3. Split our data into a training set (which we give to the model) and a test set (which the model doesn't get to see)


To keep our code organized, let's put these functionalities into **python functions**.



In [ ]:
def load_categories(dataset_path: str) -> pd.dataframe:
  """Loads all CSV files from the folder """

def data_preparation(
    dataset_path:str, categories: dict, test_size: float, random_seed: int
    ) -> tuple[tf.data.Dataset, tf.data.Dataset, tf.data.Dataset, tf.data.Dataset]:
    ...



In [8]:
print(os.listdir(path + '/Drones/15-21'))

['160.csv', '102.csv', '171.csv', '038.csv', '172.csv', '159.csv', '067.csv', '054.csv', '125.csv', '070.csv', '071.csv', '116.csv', '090.csv', '103.csv', '124.csv', '036.csv', '029.csv', '110.csv', '101.csv', '020.csv', '105.csv', '133.csv', '114.csv', '176.csv', '156.csv', '161.csv', '056.csv', '148.csv', '081.csv', '085.csv', '079.csv', '035.csv', '059.csv', '168.csv', '179.csv', '100.csv', '053.csv', '129.csv', '078.csv', '180.csv', '028.csv', '096.csv', '141.csv', '147.csv', '152.csv', '048.csv', '025.csv', '037.csv', '142.csv', '140.csv', '057.csv', '151.csv', '006.csv', '122.csv', '150.csv', '162.csv', '115.csv', '012.csv', '138.csv', '097.csv', '016.csv', '135.csv', '072.csv', '088.csv', '164.csv', '086.csv', '045.csv', '113.csv', '021.csv', '145.csv', '137.csv', '178.csv', '093.csv', '046.csv', '106.csv', '062.csv', '134.csv', '002.csv', '060.csv', '128.csv', '052.csv', '158.csv', '181.csv', '117.csv', '004.csv', '074.csv', '087.csv', '157.csv', '009.csv', '109.csv', '143.csv'